In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from bisect import bisect_left
import scipy
import seaborn as sns

#10Mb, 50Mb, 100Mb a 90 FPS per cable entre ordinadors

In [ ]:
def array_file(file_path1, file_path2, file_path3):
    def read_file(file_path):
        with open(file_path, 'r') as file:
            lines = file.read().split('\n')

        rtt_values = [float(rtt) / 1000 for rtt in lines[0].split()]
        frame_numbers = list(map(int, lines[1].split()))
        time_values = [float(time.replace(',', '.')) for time in lines[2].split()]
        total_packets = list(map(int, lines[3].split()))

        return [rtt_values, frame_numbers, time_values, total_packets]

    file1 = read_file(file_path1)
    file2 = read_file(file_path2)
    file3 = read_file(file_path3)

    return file1, file2, file3

def filter_data(data, filter_range):
    filtered_rtt = []
    filtered_time = []

    for rtt, time in zip(data[0], data[2]):
        if (filter_range - 0.1) <= time <= (filter_range + 0.1):
            filtered_rtt.append(rtt)
            filtered_time.append(time)

    return filtered_rtt, filtered_time

def plot_by_time(data_0, data_1, data_2, filter_zone):
    filtered_rtt0, filtered_time0 = filter_data(data_0, filter_zone)
    filtered_rtt1, filtered_time1 = filter_data(data_1, filter_zone)
    filtered_rtt2, filtered_time2 = filter_data(data_2, filter_zone)

    plt.figure(figsize=(12, 6))

    plt.plot(filtered_time0, filtered_rtt0, marker='o', linestyle='-', color='b', label='Data 1')
    plt.plot(filtered_time1, filtered_rtt1, marker='o', linestyle='-', color='orange', label='Data 2')
    plt.plot(filtered_time2, filtered_rtt2, marker='o', linestyle='-', color='g', label='Data 3')

    plt.xlabel('Time (s)')
    plt.ylabel('RTT (s)')
    plt.title('2D Line Plot of RTT over Time')
    plt.legend()
    plt.show()

def calculate_mean_median(values1, values2, values3):
    def calculate_mean_median(values):
        mean = (sum(values)) / len(values)
        sorted_values = sorted(values)
        n = len(sorted_values)
        mid = n // 2
        deviation = np.std(values)

        if n % 2 == 0:  # if even
            median = float((sorted_values[mid - 1] + sorted_values[mid]) / 2)
        else:  # if odd
            median = float(sorted_values[mid])

        return format(mean, '.4f'), format(median, '.4f'), format(deviation, '.4f')

    mean1, median1, std_dev1 = calculate_mean_median(values1)
    mean2, median2, std_dev2 = calculate_mean_median(values2)
    mean3, median3, std_dev3 = calculate_mean_median(values3)

    return [(mean1, median1, std_dev1), (mean2, median2, std_dev2), (mean3, median3, std_dev3)]

def box_plots(Result, Mean_median, y_length, label, titles):
  ax.boxplot([Result[0][0], Result[1][0], Result[2][0]], labels = label,showmeans = True, meanline=True)
  ax.set_title(titles)
  ax.set_ylabel('Miliseconds')
  plt.ylim(y_length[0], y_length[1])
  ax.legend(['Mean: ' + Mean_median[0][0] + ', Median: ' + Mean_median[0][1] + ", Std_dev: " + Mean_median[0][2],
            'Mean: ' + Mean_median[1][0] + ', Median: ' + Mean_median[1][1] + ", Std_dev: " + Mean_median[1][2],
            'Mean: ' + Mean_median[2][0] + ', Median: ' + Mean_median[2][1] + ", Std_dev: " + Mean_median[2][2]]
            , loc='upper right',bbox_to_anchor=(1.8, 1), fancybox=True, shadow=True, ncol=1)
  plt.show()

#CDF##############################################################################################################

def plot_cdf(result, labels, width, width_2, graph_size, titles, line_widths):
  def ecdf(a):
      x, counts = np.unique(a, return_counts=True)
      cusum = np.cumsum(counts)
      return x, cusum / cusum[-1]

  def plot_ecdf(a, label, line_width, i):
      x, y = ecdf(a)
      x = np.insert(x, 0, x[0])
      y = np.insert(y, 0, 0.)
      plt.plot(x, y, label = label[0], linewidth = line_width[i])

  plt.figure(figsize = (graph_size,6))
  plot_ecdf(result[0][0], labels, line_widths, 0)
  plot_ecdf(result[1][0], labels, line_widths, 1)
  plot_ecdf(result[2][0], labels, line_widths, 2)

  # Plot the CDF
  plt.legend(labels)
  plt.xlabel('Miliseconds')
  plt.xlim(width_2, width)
  plt.ylim(0,1)
  plt.ylabel('Cumulative Probability')
  plt.title(titles)
  plt.grid(True)
  plt.show()

def percent_packets(data1, data2, data3):
  packets1 = abs(float((data1[1] - data1[0])/ data1[0])*100)
  packets2 = abs(float((data2[1] - data2[0])/ data2[0])*100)
  packets3 = abs(float((data3[1] - data3[0])/ data3[0])*100)

  return [packets1, packets2, packets3]

In [ ]:
######################## M50b_90FPS_40MHz_1m_5m_10m UP (CORRECTEEEEE!!!!!!!!)
M50b_90FPS_40MHz_1m = ""
M50b_90FPS_40MHz_5m = ""
M50b_90FPS_40MHz_10m= ""

result_1 = array_file(M50b_90FPS_40MHz_1m, M50b_90FPS_40MHz_5m, M50b_90FPS_40MHz_10m)
mean_median = calculate_mean_median(result_1[0][0], result_1[1][0], result_1[2][0])

print(percent_packets(result_1[0][3], result_1[1][3], result_1[2][3]))

y_range = [1, 9]
labels = ['1m', '5m', '10m']
titles = "Distance comparation: 50Mbps - 90 FPS - 40Mhz"
fig, ax = plt.subplots()
box_plots(result_1, mean_median, y_range, labels, titles)

line_widths = [2, 2, 2]
plot_cdf(result_1, labels, 45, 0, 12, titles, line_widths)

#plot_by_time(result_1[0], result_1[1], result_1[2], 18)

In [ ]:
######################## M50b_30_60_90FPS_40MHz_1m (CORRECTEEEEE!!!!!!!!!!!)
data1 = ""
data2 = ""
data3 = ""

result_3 = array_file(data1, data2, data3)
mean_median = calculate_mean_median(result_3[0][0], result_3[1][0], result_3[2][0])

print(percent_packets(result_3[0][3], result_3[1][3], result_3[2][3]))

y_range = [1, 12]
labels = ['30FPS', '60FPS', '90FPS']
titles = "Bandwidth comparation: 50Mbps - 40MHz - 1m"
fig, ax = plt.subplots()
box_plots(result_3, mean_median, y_range, labels, titles)

line_widths = [2, 2, 2]
plot_cdf(result_3, labels, 12, 1, 12, titles, line_widths)

plot_by_time(result_3[0], result_3[1], result_3[2], 16)